# Linear Programming

### Task 01

There are 168 hours in a week. Say we want to allocate our time between classes and
studying (S), fun activities and going to parties (P), and everything else (E) (eating, sleeping,
taking showers, etc).

Suppose that to survive we need to spend at least 56 hours on E (8
hours/day). To maintain sanity we need P + E ≥ 70. To pass our courses, we need S ≥ 60,
but more if don’t sleep enough or spend too much time partying: 2S + E − 3P ≥ 150. (E.g.,
if don’t go to parties at all then this isn’t a problem, but if we spend more time on P then
need to sleep more or study more).

- Variables: S, P, E
- Objective: maximize 2P + E
- Subject to сonstraints: 
 * S + P + E = 168
 * E ≥ 56
 *  S ≥ 60
 *  2S + E − 3P ≥ 150
 *  P + E ≥ 70
 *  P ≥ 0 (can’t spend negative time partying)

In [36]:
from scipy.optimize import linprog

c = [-2, 0, -1]
Au = [[0, 0, -1], [-1, 0, 0], [-2, 3, -1], [0, -1, -1], [0, -1, 0]]
bu = [-56, -60, -150, -70, 0]
Ae = [[1, 1, 1]]
be = [[168]]
res = linprog(c, A_ub=Au, b_ub=bu, A_eq = Ae, b_eq = be, method='simplex')

print("Happiness is {}, with study {} , party {} and else {}".format(-res.fun, res.x[0], res.x[1], res.x[2]))

Happiness is 266.0, with study 98.0 , party 0.0 and else 70.0


### Task 2

Suppose you have 4 production plants for making cars. Each works a little differently in terms of
labor needed, materials, and pollution produced per car:


|  -  | labor | materials | pollution |
|---|---|---|---|
| plant 1 | 2 | 3 | 15 |
| plant 2 | 3 | 4 | 10 |
| plant 3 | 4 | 5 | 9 |
| plant 4 | 5 | 6 | 7 |

Suppose we need to produce at least 400 cars at plant 3 according to a labor agreement. We have
3300 hours of labor and 4000 units of material available. We are allowed to produce 12000 units of
pollution, and we want to maximize the number of cars produced. How can we model this?

x1-4 - factory car output

- Variables: x1, x2, x3, x4
- Objective: maximize x1 + x2 + x3 + x4
- Subject to сonstraints:
  * x3 >= 400
  * 2x1 + 3x2 + 4x3 + 5x4 <= 3300
  * 3x1 + 4x2 + 5x3 + 6x4 <= 4000
  * 15x1 + 10x2 + 9x3 + 7x4 <= 12000

In [52]:
c = [-1, -1, -1, -1]
Au = [[0, 0, -1, 0], [2, 3, 4, 5], [3, 4, 5, 6], [15, 10, 9, 7]]
bu = [-400, 3300, 4000, 12000]
res = linprog(c, A_ub=Au, b_ub=bu, method='simplex')

print("Maximum output is {} with \nplant 1: {}, plant 2: {},\nplant 3: {}, plant 4: {}"\
      .format(-res.fun, res.x[0], res.x[1], res.x[2], res.x[3]))

Maximum output is 1013.3333333333334 with 
plant 1: 453.33333333333337, plant 2: 160.0,
plant 3: 400.0, plant 4: 0.0


### Task 03. Modeling Network Flow

We can model the max flow problem as a linear program too. 

Variables: Set up one variable x_uv for each edge (u, v). Let’s just represent the positive flow since
it will be a little easier with fewer constraints.

- Objective:
 \begin{equation*} \max{(\sum_u x_{ut} − \sum_u x_{tu})} - \text{maximize the flow into t minus any flow out of t}\end{equation*}
- Constraints:
    \begin{equation*} \forall (u, v): 0 ≤ x_{uv} ≤ c(u, v) - \text{capacity constraints}\end{equation*}
    \begin{equation*} \forall v \in \{s, t\}: \sum_{u} {x_{uv}} = \sum_{u} {x_{vu}} - \text{flow conservation} \end{equation*}
    
    ![graph_01](img/lp_1_graph.png)

Resulting LP is

- Maximize: x_ct + x_dt
- Variables x_sa, x_sb, x_ac, x_bc, x_cb, x_bd, x_ct, x_dt
- Subject to constraints
   * 0 ≤ x_sa ≤ 4, 0 ≤ x_sb ≤ 2, 0 ≤ x_ac ≤ 3, 0 ≤ x_bc ≤ 2,
   * 0 ≤ x_cb ≤ 1, 0 ≤ x_bd ≤ 3, 0 ≤ x_ct ≤ 2, 0 ≤ x_dt ≤ 4
   * x_sa = x_ac, x_sb + x_cb = x_bc + x_bd, x_ac + x_bc = x_cb + x_ct, x_bd = x_dt.

In [67]:
# [x_sa, x_sb, x_ac, x_bc, x_cb, x_bd, x_ct, x_dt]
# [ 1  , 2   , 3   , 4   , 5   , 6   , 7   , 8   ]
c = [0, 0, 0, 0, 0, 0, -1, -1]
Au = [
    # positivity 
    [-1, 0, 0, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0, 0, 0], [0, 0, -1, 0, 0, 0, 0, 0], [0, 0, 0, -1, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, -1, 0], [0, 0, 0, 0, 0, 0, 0, -1],
    # capacity
    [1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1]]
bu = [
    # positivity    
    0, 0, 0, 0,
    0, 0, 0, 0,
    # capacity
    4, 2, 3, 2,
    1, 3, 2, 4
     ]
Ae = [[1, 0, -1, 0, 0, 0, 0, 0], [0, 1, 0, -1, 1, -1, 0, 0], [0, 0, 1, 1, -1, 0, -1, 0], [0, 0, 0, 0, 0, 1, 0, -1]]
be = [0, 0, 0, 0]
res = linprog(c, A_ub=Au, b_ub=bu, A_eq = Ae, b_eq = be, method='simplex')

print("Maximum output is {} with {}"\
      .format(-res.fun, res.x))

Maximum output is 5.0 with [3. 2. 3. 0. 1. 3. 2. 3.]


### 2-Player Zero-Sum Games